##Pacotes

In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Imports

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
import pickle
#from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import torch
import pprint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "drive/MyDrive/Mestrado/defesa/data/70-15-15"
model_path = "drive/MyDrive/Mestrado/defesa/models"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = load_dataset("csv", data_files={"train":data_path+'/comunic_treino.csv', "test": data_path+'/comunic_teste.csv', "validation":data_path+'/comunic_valid.csv'})

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

def preprocess_data(docs):
    text = docs["text"]
    return tokenizer(docs["text"], padding='max_length', max_length=512, truncation=True)

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True) #,remove_columns=dataset['train'].column_names)

In [ ]:
encoded_dataset.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=13)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [ ]:
batch_size = 16
metric_name = "f1"

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    f"bert-finetuned-comunicados",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 5,
    learning_rate=1.8e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.4,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    logging_steps=5000,
    logging_dir='./logs',
)


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score, hamming_loss
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}    

In [ ]:
from torch import nn
from transformers import Trainer

# the model is waiting for "labels", not "label"
encoded_dataset = encoded_dataset.rename_column("label", "labels")

# Get training encoded data into DataFrame
df = encoded_dataset["train"].to_pandas()

# Get a vector of weights (lowest weight for majority class)
class_weights = (1 - (df["labels"].value_counts().sort_index() / len(df))).values
print(class_weights)

# Put the weights vector to cuda as a Pytorch tensor
class_weights = torch.from_numpy(class_weights).float().to("cuda")

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # define loss function with calss weights
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        # Compute loss
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss


[0.9372237  0.92838196 0.98496905 0.96905393 0.52961981 0.96374889
 0.98231653 0.94341291 0.98585323 0.90981432 0.97259063 0.98231653
 0.9106985 ]


In [ ]:
trainer = CustomTrainer(
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
#trainer.train()

n_folds = 5
kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Define a function to perform a single fold of training and evaluation
def run_fold(train_dataset, eval_dataset, fold):
    print(f"Training fold {fold}")
    trainer = Trainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,         
        eval_dataset=eval_dataset,           
        compute_metrics=compute_metrics      
    )
    trainer.train()
    print(f"Evaluating fold {fold}")
    trainer.evaluate()


In [ ]:
# Perform k-fold cross-validation

history={'accuracy':[],'precision':[],'recall':[],'f1':[]}
splits=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset['train'])))):
    print('Fold: ',fold+1)
    train_sampler=dataset["train"].select(train_idx)
    val_sampler=dataset["train"].select(val_idx)
    train_dataset = train_sampler.map(preprocess_data, batched=True,remove_columns=dataset['train'].column_names)
    train_dataset.set_format("torch")
    val_dataset=val_sampler.map(preprocess_data, batched=True,remove_columns=dataset['train'].column_names)
    val_dataset.set_format("torch")
    model = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=13)
    model.resize_token_embeddings(len(tokenizer))
    trainer = CustomTrainer( model,
      training_args,
      train_dataset=encoded_dataset["train"],
      eval_dataset=encoded_dataset["validation"],
      tokenizer=tokenizer,
      compute_metrics=compute_metrics, 
      callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
    )
    trainer.train()
    metrics=trainer.evaluate(eval_dataset=encoded_dataset['test'])
    print("Métricas dataset teste em ",fold+1,": ")
    pprint.pprint(metrics)
    history['precision'].append(metrics['eval_precision'])
    history['recall'].append(metrics['eval_recall'])
    history['f1'].append(metrics['eval_f1'])
    history['accuracy'].append(metrics['eval_accuracy'])

    del model


Fold:  1


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.428904,0.687243,0.584627,0.687243,0.614796
2,No log,0.997216,0.786008,0.784534,0.786008,0.753790
3,No log,0.841551,0.810700,0.774417,0.810700,0.786671
4,No log,0.756762,0.831276,0.824762,0.831276,0.820122
5,No log,0.722548,0.835391,0.835487,0.835391,0.824447
6,No log,0.732632,0.855967,0.857840,0.855967,0.849446
7,No log,0.734926,0.864198,0.864940,0.864198,0.858698
8,No log,0.722990,0.855967,0.861046,0.855967,0.853036
9,No log,0.725270,0.860082,0.861831,0.860082,0.856062
10,No log,0.730486,0.855967,0.857994,0.855967,0.850427


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Pr

Métricas dataset teste em  1 : 
{'epoch': 10.0,
 'eval_accuracy': 0.8395061728395061,
 'eval_f1': 0.8253969008143089,
 'eval_loss': 0.7888363003730774,
 'eval_precision': 0.8445719461151561,
 'eval_recall': 0.8395061728395061,
 'eval_runtime': 8.5644,
 'eval_samples_per_second': 28.373,
 'eval_steps_per_second': 1.868}
Fold:  2


...

Map:   0%|          | 0/226 [00:00<?, ? examples/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.554890,0.625514,0.493714,0.625514,0.536747
2,No log,1.062443,0.781893,0.727887,0.781893,0.738993
3,No log,0.891670,0.806584,0.785819,0.806584,0.784683
4,No log,0.790731,0.823045,0.806856,0.823045,0.802789
5,No log,0.731774,0.818930,0.822704,0.818930,0.814174
6,No log,0.724035,0.839506,0.830718,0.839506,0.830089
7,No log,0.700627,0.847737,0.840996,0.847737,0.840618
8,No log,0.712751,0.847737,0.845079,0.847737,0.841162
9,No log,0.706047,0.860082,0.858549,0.860082,0.855451
10,No log,0.711649,0.860082,0.857823,0.860082,0.854644


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Pr

Métricas dataset teste em  5 : 
{'epoch': 10.0,
 'eval_accuracy': 0.8641975308641975,
 'eval_f1': 0.8575576940604206,
 'eval_loss': 0.7591276168823242,
 'eval_precision': 0.862204532913541,
 'eval_recall': 0.8641975308641975,
 'eval_runtime': 8.5363,
 'eval_samples_per_second': 28.467,
 'eval_steps_per_second': 1.874}


In [ ]:
avg_precision=np.mean(history['precision'])
avg_recall=np.mean(history['recall'])
avg_f1=np.mean(history['f1'])
avg_accuracy=np.mean(history['accuracy'])
print("***** Resultados Cross Validation *******")
print('Precision:',avg_precision)
print('Recall:',avg_recall)
print('F1 Score:',avg_f1)
print('Accuracy:',avg_accuracy)

***** Resultados Cross Validation *******
Precision: 0.858678015553864
Recall: 0.8592592592592592
F1 Score: 0.8511255354111983
Accuracy: 0.8592592592592592
